## **Prepare Dataset**

### Download Dataset

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"s00hyun00","key":"521460db8cc76b0d510d56c9bd51eb71"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list -s german

ref                                                     title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
uciml/german-credit                                     German Credit Risk                                  11KB  2016-12-14 21:25:02          41685        372  0.8235294        
meowmeowmeowmeowmeow/gtsrb-german-traffic-sign          GTSRB - German Traffic Sign Recognition Benchmark  612MB  2018-11-25 18:12:34          83235        968  0.8235294        
sterby/german-recipes-dataset                           German Recipes Dataset                               5MB  2019-03-06 16:25:22           3999         66  1.0              
mpwolke/cusersmarildownloadsgermancsv                   Creditability - German Credit Data               

In [ ]:
! kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

 99% 606M/612M [00:04<00:00, 213MB/s]
100% 612M/612M [00:04<00:00, 148MB/s]


In [ ]:
!unzip gtsrb-german-traffic-sign.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/5/00005_00053_00010.png  
  inflating: train/5/00005_00053_00011.png  
  inflating: train/5/00005_00053_00012.png  
  inflating: train/5/00005_00053_00013.png  
  inflating: train/5/00005_00053_00014.png  
  inflating: train/5/00005_00053_00015.png  
  inflating: train/5/00005_00053_00016.png  
  inflating: train/5/00005_00053_00017.png  
  inflating: train/5/00005_00053_00018.png  
  inflating: train/5/00005_00053_00019.png  
  inflating: train/5/00005_00053_00020.png  
  inflating: train/5/00005_00053_00021.png  
  inflating: train/5/00005_00053_00022.png  
  inflating: train/5/00005_00053_00023.png  
  inflating: train/5/00005_00053_00024.png  
  inflating: train/5/00005_00053_00025.png  
  inflating: train/5/00005_00053_00026.png  
  inflating: train/5/00005_00053_00027.png  
  inflating: train/5/00005_00053_00028.png  
  inflating: train/5/00005_00053_00029.png  
  inflating: train/5/00005_00054_00000.png  
  in

In [ ]:
import pandas as pd

train = pd.read_csv('./Train.csv')
test = pd.read_csv('./Test.csv')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39209 entries, 0 to 39208
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Width    39209 non-null  int64 
 1   Height   39209 non-null  int64 
 2   Roi.X1   39209 non-null  int64 
 3   Roi.Y1   39209 non-null  int64 
 4   Roi.X2   39209 non-null  int64 
 5   Roi.Y2   39209 non-null  int64 
 6   ClassId  39209 non-null  int64 
 7   Path     39209 non-null  object
dtypes: int64(7), object(1)
memory usage: 2.4+ MB


## Pre-process Dataset

In [ ]:
num_classes = len(train['ClassId'].unique())
print("Number of classes: ", num_classes)

Number of classes:  43


In [ ]:
data_dir = './'
train_path = './Train'
test_path = './Test'

# Resizing the images to 30x30x3
IMG_HEIGHT = 30
IMG_WIDTH = 30
channels = 3

# Label Overview
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

## **Wandb.ai - Hyperparameter Tuning** 

In [ ]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.1/203.1 kB 23.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
sweep_config = {
    'method': 'random',
    'metric': {'goal': 'minimize', 'name': 'test_loss'},
    'parameters': {
        'optimizer': {'values': ['adam', 'sgd']},
        'learning_rate': {'distribution': 'uniform', 'max': 0.001, 'min': 0.0001},
        'epochs': {'value': 15}
    }
}

sweep_id = wandb.sweep(sweep_config, entity="atds_ml", project="resnet50")

Create sweep with ID: k3k9bdxd
Sweep URL: https://wandb.ai/atds_ml/resnet50/sweeps/k3k9bdxd


In [ ]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import os
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(config=None):

    with wandb.init(config=config):
        config = wandb.config

        loader = build_dataset(data_dir)
        network = build_network()
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            wandb.log({"loss": avg_loss, "epoch": epoch, "optimizer": config.optimizer, "learning_rate": config.learning_rate})     

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import torch
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from torch.utils.data import TensorDataset 

def build_dataset(data_dir):
  image_data = []
  image_labels = []

  for i in range(num_classes):
    path = data_dir + '/Train/' + str(i)
    images = os.listdir(path)

    for img in images:
      try:
        image = cv2.imread(path + '/' + img)
        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
        image_data.append(np.array(resize_image))
        image_labels.append(i)
      except:
        print("Error in " + img)
        
  # Changing the list to numpy array
  image_data = torch.FloatTensor(np.array(image_data))
  image_data = np.transpose(image_data,(0,3,1,2))
  image_labels = torch.FloatTensor(np.array(image_labels))

  X_train, X_val, y_train, y_val = train_test_split(image_data, image_labels, test_size=0.2, random_state=42, shuffle=True)

  X_train = X_train/255 
  X_val = X_val/255

  print("X_train.shape", X_train.shape)
  print("X_valid.shape", X_val.shape)
  print("y_train.shape", y_train.shape)
  print("y_valid.shape", y_val.shape)

  y_train = torch.FloatTensor(keras.utils.to_categorical(y_train, num_classes))
  y_val = torch.FloatTensor(keras.utils.to_categorical(y_val, num_classes))

  train_dataset = TensorDataset(X_train, y_train)
  val_dataset = TensorDataset(X_val, y_val)

  dataloaders = {'train': torch.utils.data.DataLoader(train_dataset, batch_size=64,shuffle=True, num_workers=2),
               'val': torch.utils.data.DataLoader(val_dataset, batch_size=64,shuffle=True, num_workers=2)}

  return dataloaders

In [ ]:
import torchvision
from torchvision import models
import torch
import torch.nn as nn

def build_network():
    model = models.resnet50(pretrained=True)
    # Freeze weights of all layers
    for param in model.parameters():
      param.requires_grad = False
    # Add new fully-connected layer with output dimension equal to number of classes
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 43)
    # Initialize weights for new layer
    nn.init.kaiming_normal_(model.fc.weight, mode='fan_out', nonlinearity='relu')
    nn.init.zeros_(model.fc.bias)

    return model.to(device)

In [ ]:
def build_optimizer(model, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(model.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(model.parameters(),
                               lr=learning_rate)
    return optimizer

In [ ]:
import torch.nn.functional as F

def train_epoch(model, dataloaders, optimizer):
    model.train()
    running_loss = 0.0
    total_samples = 0

    for inputs, labels in dataloaders['train']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = F.cross_entropy(outputs, torch.max(labels, 1)[1])

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        total_samples += inputs.size(0)

    avg_loss = running_loss / total_samples
    return avg_loss

###**Testing different optimizers and learning rates**

In [ ]:
wandb.agent(sweep_id, train, count=20)

wandb: Agent Starting Run: j04pmq1f with config:
wandb: 	epochs: 15
wandb: 	learning_rate: 0.0004798686857326141
wandb: 	optimizer: adam
wandb: Currently logged in as: sk7905 (atds_ml). Use `wandb login --relogin` to force relogin


X_train.shape torch.Size([31367, 3, 30, 30])
X_valid.shape torch.Size([7842, 3, 30, 30])
y_train.shape torch.Size([31367])
y_valid.shape torch.Size([7842])


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 249MB/s]


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▅▄▃▂▂▂▂▂▁▁▁▁▁▁
epoch,14
learning_rate,0.00048
loss,2.06947
optimizer,adam


wandb: Agent Starting Run: bn0ucvbc with config:
wandb: 	epochs: 15
wandb: 	learning_rate: 0.0005468656899907587
wandb: 	optimizer: sgd


...

X_train.shape torch.Size([31367, 3, 30, 30])
X_valid.shape torch.Size([7842, 3, 30, 30])
y_train.shape torch.Size([31367])
y_valid.shape torch.Size([7842])


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▄▃▃▂▂▂▂▂▁▁▁▁▁▁
epoch,14
learning_rate,0.0008
loss,1.74479
optimizer,adam


wandb: Agent Starting Run: ah23ibqo with config:
wandb: 	epochs: 15
wandb: 	learning_rate: 0.0002848380458173692
wandb: 	optimizer: adam


X_train.shape torch.Size([31367, 3, 30, 30])
X_valid.shape torch.Size([7842, 3, 30, 30])
y_train.shape torch.Size([31367])
y_valid.shape torch.Size([7842])


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▅▄▃▃▂▂▂▂▁▁▁▁▁▁
epoch,14
learning_rate,0.00028
loss,2.58922
optimizer,adam
